### 第五章 Monte Carlo Methods 蒙特卡洛方法

这一章蒙特卡洛（简称“MC”）方法的本质也是在寻找最佳策略。但与之前的动态规划（简称“DP”）方法不同的是，这个方法不需要具备环境知识。

即蒙特卡洛方法 __仅需通过采样（即状态、动作、收益的序列）的平均回报__，来解决强化学习问题。

_目前术语“蒙特卡洛”或“MC”泛指任何包含大量随机成分的估计方法。_

#### 5.1 Monte Carlo Prediction 蒙特卡洛预测

__核心思想:__

- __期望回报__ - 从某个状态开始时的回报+未来期望回报的累积。
- __随着观察到更多的回报，平均值应该收敛到期望值__ —— 这个想法是所有蒙特卡洛方法的基础。
- 每次在一个回合（Episode）中出现状态的情况称为对该状态的一次 __访问__。

__两种实现方法:__

1.	__首次访问 (First-visit):__ 这是在一个回合中第一次访问的时刻。首次访问蒙特卡罗 (MC) 方法将估计值定义为首次访问后所有回报的平均值。——本节主要讨论这个方法。
2.	__每次访问 (Every-visit)：__ 每次访问蒙特卡罗 (MC) 方法会对所有访问后的回报进行平均，得到估计值。—— 这个更像是一个函数的参数近似，在以后章节中展开。




![first-visit](img.png)

1.	书中给出的实现 __首次访问 (First-visit)__ 的逻辑。 


#### 5.2 Monte Carlo Estimation of Action Values 动作价值的蒙特卡洛估计

简单来说，这节引入了概念：计算动作的价值（“状态-动作”二元组的价值，也即“状态-动作”二元组）比起计算状态的价值更加有用一些。

下面的5.3章介绍了如何通过蒙特卡洛ES算法来实现 $Q{*}(s, a)$ 近似的。

#### 5.3 Monte Carlo Control 蒙特卡洛控制 （有试探性的）

这章讲了如何利用蒙特卡洛方法来解决控制问题，即如何近似最优的策略 ${\pi}$。

- 在每个回合(Episode)之后，观测到的回报用于策略评估，然后在回合中访问到的所有状态上改进策略。
- __Monte Carlo ES__ (Exploring Stares) 算法:

![MCES](img_1.png)

上面的蒙特卡洛ES算法：每个 __状态-动作__ 二元祖对的所有回报都被累积并取平均值，而不考虑它们被观测时的策略是什么。

蒙特卡罗 ES 无法收敛到任何次优策略。只有当策略和价值函数都达到最优时，系统才能达到稳定。

蒙特卡洛ES是指有试探性的控制，确保在策略评估和改进过程中，每个状态-动作对都有机会被探索到。

#### 5.4 Monte Carlo Control without Exploring Starts 没有试探性出发假设的蒙特卡洛控制

这章介绍了一些新的概念：

- __On-policy 同轨方法__ 和 __Off-policy 离轨方法__
    - On-policy 同轨方法 使用和当前正在执行的策略进行评估和改进。包括后面几章要介绍的SARSA。
    - Off-policy 离轨方法 使用行为策略进行探索，但评估和改进的是目标策略。例子包括 Q-learning。
    - 这两种方法各有优缺点，On-policy 方法稳定而可靠，但探索效率可能不高，而 Off-policy 方法灵活高效，但收敛性和稳定性需要较难实现（方差较大）。
- __Soft-policy 软策略方法__：
    - 策略在每个状态下会以一定的概率选择不同的行动，而不是总是选择唯一的最佳行动，即 __保持一定随机性__。
    -  $\epsilon$-贪婪策略：$\epsilon$-贪婪策略以一定的小概率 $\epsilon$ 随机选择一个动作进行探索（即选择任意可能的行为），而以 $1-\epsilon$ 的概率选择当前最优的行为。

__On-policy first-visit MC control (for $\epsilon$-soft policies)__:

![On-policy first-visit MC control](img_2.png)


#### 5.5  Off-policy Prediction via Importance Sampling 基于重要度采样的离轨策略

离轨策略的核心思想是采用两个策略：

- __Target Policy 目标策略__：我们希望最终学习到的策略，即我们要优化的策略。使用目标策略进行评估和改进。目标策略通常是为了最大化未来的累积奖励。
- __Behavior Policy 行动策略__：是实际用来与环境交互、采集经验的策略。行动策略用于生成数据样本，即用来收集状态-动作-奖励的序列。与目标策略不同，行动策略的设计可以更偏向于探索，以便能够覆盖更广的状态和动作空间。

通过将这两种策略分开，离轨策略方法可以在不受限制地探索的同时，专注于改进目标策略。这使得它在许多复杂环境下特别有效，例如需要探索多个可能的路径或需要通过大规模经验数据进行优化的场景。

__Q-learning__ 就是典型的离轨策略方法。在 Q-learning 中，行为策略通常以贪婪方式进行探索，允许随机选择动作以保持对环境的探索，而目标策略则选择具有最高 Q 值的动作，以改进政策。

而几乎所有的离轨策略都使用了 __Importance Sampling 重要度采样__ ：通过根据目标策略和行为策略下轨迹发生的 _相对概率对回报进行加权_ ，这个权重被称为 _重要性采样比率_ 。

##### 给定起始状态 ${S_t}$ 后续的状态-动作轨迹在策略 ${\pi}$ 下发生的概率是：

$$
\Pr\{A_t, S_{t+1}, A_{t+1}, \ldots, S_T \mid S_t, A_{t:T-1} \sim \pi \} 
= \pi(A_t \mid S_t) p(S_{t+1} \mid S_t, A_t)\pi(A_{t+1} \mid S_{t+1})  \cdots p(S_T \mid S_{T-1}, A_{T-1})
= \prod_{k=t}^{T-1} \pi(A_k \mid S_k) p(S_{k+1} \mid S_k, A_k)
$$

在目标策略和行动策略轨迹下的相对概率 (__重要度采样比__) 是：

$$
\rho_{t:T-1} = \frac{\prod_{k=t}^{T-1} \pi(A_k \mid S_k) p(S_{k+1} \mid S_k, A_k)}{\prod_{k=t}^{T-1} b(A_k \mid S_k) p(S_{k+1} \mid S_k, A_k)}
= \prod_{k=t}^{T-1} \frac{\pi(A_k \mid S_k)}{b(A_k \mid S_k)}
$$

这里的 $b$ 是行为策略。在公式中，$b(A_k \mid S_k)$ 表示在行为策略 $b$ 下，在状态 $S_k$ 选择动作 $A_k$ 的概率，而 $\pi(A_k \mid S_k)$ 则表示目标策略下在状态 $S_k$ 选择动作 $A_k$ 的概率。通过重要性采样的方式，我们能够使用行为策略 $b$ 采集的数据来估计和改进目标策略 $\pi$，从而实现有效的策略学习。

这里的推导目的是想说明：__重要度采样比只与两个策略和样本序列数据相关，而与 MDP 的动态特性 (状态转移概率) 无关。__

- __Ordinary importance sampling 平均重要度采样：__

    - $\mathcal{T}(s)$ 表示所有访问状态 $s$ 的时间步集合；
    - ${T}(t)$ 表示时间 $t$ 之后的首次终止时间；
    - $\rho_{t:T(t)-1}$ 是重要度采样比；
    - $G(t)$ 表示从时间 $t$ 到 $\mathcal{T}(t)$ 之间的回报。

$$
V(s) \doteq \frac{\sum_{t \in \mathcal{T}(s)} \rho_{t:T(t)-1} G_t}{|\mathcal{T}(s)|}
$$

	
- __Weighted importance sampling 加权重要度采样：__

    - 分子部分：$\sum_{t \in \mathcal{T}(s)} \rho_{t:T(t)-1} G_t$ 是对状态 $s$ 在所有时间步的回报 $G_t$ 进行加权求和，其中 $\rho_{t:T(t)-1}$ 是重要性采样比率，用于调整由于行为策略与目标策略不同所导致的偏差。
    - 分母部分：$\sum_{t \in \mathcal{T}(s)} \rho_{t:T(t)-1}$ 是所有时间步的加权和，用于归一化计算。这一部分用于保证加权之后的期望计算合理，相当于调整权重后的计数。

$$
V(s) \doteq \frac{\sum_{t \in \mathcal{T}(s)} \rho_{t:T(t)-1} G_t}{\sum_{t \in \mathcal{T}(s)} \rho_{t:T(t)-1}}
$$

![MSE](img_3.png)


#### 5.6 Incremental Implementation 增量式实现

所谓“增量式”指的是把每次Episode后得到的回报 $G$ 通过增量式方法来累积，而非简单的像上面 __平均重要度采样__ 那样单求平均值。

增量式方法也需要同时考虑 _同轨策略_ 和 _离轨策略_。单这里我们只讨论更加广泛使用的 _离轨策略_：

- 增量加权求和的更新方法是（其中，$C_n$ 是前几个回报所赋权重的累积和）：

$$
V_{n+1} \doteq \frac{\sum_{k=1}^n W_k G_k}{\sum_{k=1}^n W_k} = V_n + \frac{W_n}{C_n} (G_n - V_n)
$$



原书中给出了一个完整的 __蒙特卡洛策略评估的逐幕增量算法__ 的实现逻辑：

![MCPINCREMENT](img_4.png)

#### 5.7 Off-policy Monte Carlo Control 离轨策略蒙特卡洛控制

这章对应的是《5.3 Monte Carlo Control 蒙特卡洛控制》里所引出介绍的其实是“同轨策略”。那由于更多实践其实是使用离轨策略的，所以这章展开来讲。

_为了探索所有可能性，离轨蒙特卡罗控制方法要求行为策略是Soft-policy软策略（即在所有状态下对所有动作的选择概率都为非零）。_

书中给出的算法实现：

![Off-policy Monte Carlo Control](img_5.png)

#### 5.8 *Discounting-aware Importance Sampling *折扣敏感的重要度采样
#### 5.9 *Per-decision Importance Sampling *每次决策型重要度采样

这两章节里讨论的主要是如何减少离轨策略估计的方差。主要是针对很长的一局(Epsilon)、且很小的 $\gamma$ 的情况下，如何调整权重参数。


### 5.10 本章总结

本章介绍了从“多幕采样数据”的经验中学习价值函数和最优策略的蒙特卡洛（MC）方法。相比于上一章的DP方法，MC有以下四个优势：

1. 不需要知道环境信息
2. 多Episode采样
3. 可以很简单和高效地聚焦于状态的一个小的子集，可以只评估关注的区域而不评估其余的状态
4. 马尔可夫性不成立时性能损失较小


##### 核心思想：从完整经验中学习

蒙特卡洛方法的核心是通过采样完整的经验轨迹来估计价值函数或找到最优策略。这是它区别于动态规划DP和时序差分TD（下一章中讲解）方法的关键特征。

##### 最重要的应用：策略评估和策略改进

__策略评估__：估计给定策略下的状态价值或动作价值函数。
__策略改进__：基于当前的价值估计来改进策略。

##### 关键算法：蒙特卡洛控制

蒙特卡洛控制算法结合了策略评估和策略改进，是寻找最优策略的重要方法。

##### 重要概念：

_a) 首次访问 vs 每次访问：_

首次访问方法只考虑每个情节中状态的首次出现。

每次访问方法考虑状态的每次出现。

这两种方法都能收敛到真实价值，但在实际应用中可能有不同的效率。

_b) 同轨(On-policy) vs 离轨(Off-policy)：_

同轨方法评估和改进用于生成数据的相同策略。

离轨方法允许评估和改进与生成数据的策略不同的策略，提供了更大的灵活性。

_c) 增量实现：_

允许算法在线学习，不需要存储所有经验就能更新估计，这在实际应用中非常重要。

##### 最重要的理论基础：大数定律

蒙特卡洛方法的有效性基于大数定律，即随着样本数量的增加，估计会收敛到真实值。

##### 总结：

同轨和离轨方法的选择，以及增量实现的应用，对于实际问题的解决尤为重要。



In [12]:
""" 
5.1 章中的用Gymnasium库来模拟21点游戏环境

- 定义了一个简单的策略：当玩家总和为20或21时停牌，否则要牌。
- 实现了蒙特卡洛预测算法，用于估计给定策略下的状态价值函数。
- 创建了一个3D表面图来可视化状态价值函数，分别展示了有可用A和无可用A的情况。
"""
import numpy as np
import gymnasium as gym
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 创建21点游戏环境
env = gym.make("Blackjack-v1", sab=True)

# 定义策略：玩家总和为20或21时停牌，否则要牌
def policy(player_curr_sum: int) -> int:
    return 0 if player_curr_sum >= 20 else 1

# 蒙特卡洛预测算法
def mc_prediction(num_episodes=500000, gamma=1.0):
    # 初始化状态价值函数
    V = np.zeros(shape=(22, 11, 2))  # (玩家总和, 庄家明牌, 是否有可用A)

    # 初始化回报和计数
    Returns = np.zeros_like(V, dtype=np.float32)
    counts = np.zeros_like(V, dtype=np.int16)

    for _ in range(num_episodes):
        # 重置环境
        state, _ = env.reset()
        terminated = False

        trajectory = []
        seen = set()

        # 生成一个轨迹
        while not terminated:
            action = policy(state[0])
            next_state, reward, terminated, _, _ = env.step(action)
            trajectory.append((state, action, reward))
            state = next_state

        # 计算回报并更新价值函数
        G = 0.0
        for state, _, reward in reversed(trajectory):
            G = gamma * G + reward
            if state not in seen:
                seen.add(state)
                Returns[state] += G
                counts[state] += 1
                V[state] = Returns[state] / counts[state]

    return V

# 可视化函数
def plot_value_function(V, title):
    x = np.arange(1, 11)
    y = np.arange(12, 22)
    X, Y = np.meshgrid(x, y)

    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=('有可用A', '无可用A'),
                        specs=[[{'type': 'surface'}, {'type': 'surface'}]])

    for i, ace in enumerate([1, 0]):
        Z = V[12:22, 1:11, ace]

        fig.add_trace(
            go.Surface(z=Z, x=X, y=Y, colorscale='Viridis',
                       colorbar=dict(title='价值', len=0.5, y=0.5-0.25*i, thickness=10)),
            row=1, col=i+1
        )

    fig.update_layout(title_text=title, autosize=False,
                      width=1000, height=500,
                      scene = dict(
                          xaxis_title='庄家明牌',
                          yaxis_title='玩家总和',
                          zaxis_title='价值'),
                      scene2 = dict(
                          xaxis_title='庄家明牌',
                          yaxis_title='玩家总和',
                          zaxis_title='价值'))

    fig.show()

# 运行蒙特卡洛预测算法
num_episodes = 500_000
V = mc_prediction(num_episodes)

# 可视化结果
plot_value_function(V, f'21点游戏状态价值函数 (迭代 {num_episodes} 次)')

In [11]:
"""
5.5章 代码实现了21点游戏的蒙特卡洛重要性采样算法。主要功能包括：

- 定义了目标策略和行为策略。
- 实现了蒙特卡洛重要性采样算法（Importance Sampling），用于估计给定起始状态的价值。
- 可视化图展现普通重要性采样和加权重要性采样的均方误差随回合数的变化。
"""
"""
【注意】：

这段代码总采样次数为 20轮 * 10,000回合 = 200,000 次内循环，串行CPU上运行时间较长，> 1小时。。

"""
import numpy as np
import gymnasium as gym
import plotly.graph_objects as go
from typing import Any
from tqdm import tqdm

# 创建21点游戏环境
env = gym.make('Blackjack-v1', sab=True)

# 目标策略：玩家总和大于等于20时停牌，否则要牌
def prob_target_policy(player_sum: int, action: int) -> float:
    if player_sum >= 20 and action == 0:
        return 1.0
    if player_sum < 20 and action == 1:
        return 1.0
    return 0.0

# 行为策略：随机选择动作
def prob_behavior_policy() -> float:
    return 0.5

# 获取随机动作
def get_action() -> int:
    return np.random.choice([0, 1])

# 计算均方误差
def mse(val_1: float, val_2: float) -> float:
    return np.sqrt((val_1 - val_2) ** 2)

# 初始化到预设的起始状态
def init_start_state(env: Any, start_state: tuple) -> tuple:
    observation, _ = env.reset()
    while tuple(observation) != start_state:
        observation, _ = env.reset()
    return observation

# 蒙特卡洛重要性采样算法
def monte_carlo_importance_sampling(total_rounds: int, episodes_per_round: int) -> dict:
    start_state = (13, 2, 1)  # 玩家总和=13，庄家明牌=2，有可用A
    target_val = -0.27726  # 书中提供的目标值
    gamma = 1.0

    value_hist = {
        'ordinary': np.zeros((total_rounds, episodes_per_round)),
        'weighted': np.zeros((total_rounds, episodes_per_round))
    }

    for r in range(total_rounds):
        print(f'正在进行第 {r+1} 轮采样')
        V_ord, V_wei, rho, count = 0, 0, 0, 0

        for t in tqdm(range(episodes_per_round)):
            terminated = False
            state = init_start_state(env, start_state)
            action = get_action()

            trajectory = []
            while not terminated:
                next_state, reward, terminated, _, _ = env.step(action)
                trajectory.append((tuple(state), action, reward))
                state = next_state
                action = get_action()

            G, W = 0, 1.0

            for state, action, reward in reversed(trajectory):
                G = gamma * G + reward
                prob_pi = prob_target_policy(state[0], action)
                prob_behav = prob_behavior_policy()
                W *= (prob_pi / prob_behav)

                if state == start_state:
                    V_ord += W * G
                    count += 1
                    V_wei += W * G
                    rho += W

            value_hist['weighted'][r, t] = mse(V_wei / rho if rho != 0 else 0, target_val)
            value_hist['ordinary'][r, t] = mse(V_ord / count, target_val)

    return value_hist

# 可视化结果
def plot_result(value_hist: dict) -> None:
    ord_hist = value_hist['ordinary'].mean(axis=0)
    weighted_hist = value_hist['weighted'].mean(axis=0)

    fig = go.Figure()

    x = np.arange(1, len(ord_hist) + 1)

    fig.add_trace(go.Scatter(x=x, y=weighted_hist, mode='lines', name='加权重要性采样',
                             line=dict(color='tomato', width=2)))
    fig.add_trace(go.Scatter(x=x, y=ord_hist, mode='lines', name='普通重要性采样',
                             line=dict(color='lightseagreen', width=2)))

    fig.update_layout(
        title='21点游戏 - 蒙特卡洛重要性采样',
        xaxis_title='回合数 (对数刻度)',
        yaxis_title='均方误差 (20次运行的平均值)',
        xaxis_type='log',
        xaxis=dict(tickmode='array', tickvals=[1, 10, 100, 1000, 10000],
                   ticktext=['1', '10', '100', '1000', '10,000']),
        yaxis=dict(range=[0, 2]),
        legend=dict(x=0.7, y=0.98),
        width=800,
        height=500
    )

    fig.show()

# 运行蒙特卡洛重要性采样算法
total_rounds = 20
episodes_per_round = 10000
value_hist = monte_carlo_importance_sampling(total_rounds, episodes_per_round)

# 可视化结果
plot_result(value_hist)

正在进行第 1 轮采样


100%|██████████| 10000/10000 [04:21<00:00, 38.20it/s]


正在进行第 2 轮采样


100%|██████████| 10000/10000 [04:14<00:00, 39.28it/s]


正在进行第 3 轮采样


...

100%|██████████| 10000/10000 [04:11<00:00, 39.81it/s]


正在进行第 19 轮采样


100%|██████████| 10000/10000 [04:09<00:00, 40.08it/s]


正在进行第 20 轮采样


100%|██████████| 10000/10000 [04:17<00:00, 38.79it/s]
